# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather_data_to_load = "cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head(20)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kapaa,22.0752,-159.3190,79.12,83,90,1.01,US,1627139215
1,norman wells,35.2226,-97.4395,86.92,70,1,17.27,US,1627139227
2,rikitea,-23.1203,-134.9692,68.61,55,0,3.51,PF,1627139026
3,saldanha,-33.0117,17.9442,60.28,54,11,6.08,ZA,1627139153
4,anibongan,7.3933,125.7189,79.54,87,42,4.16,PH,1627139227
5,cherskiy,68.7500,161.3000,37.90,87,69,4.23,RU,1627139033
6,ishigaki,24.3448,124.1572,83.52,85,100,22.03,JP,1627139227
7,sao joao da barra,20.5113,76.2129,76.98,80,100,13.13,IN,1627139228
8,bulgan,48.8125,103.5347,59.47,83,100,1.30,MN,1627139228
9,leningradskiy,69.3833,178.4167,36.18,95,100,2.93,RU,1627139228


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
#configure gmaps
gmaps.configure(api_key=g_key)

#locations
locations=dropna_weather_data[['Lat', 'Lng']]
humidity=dropna_weather_data['Humidity'].astype(float)

In [35]:
# Plot Heatmap
fig=gmaps.figure()

#create Heat Layer
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

#Add Layer
fig.add_layer(heat_layer)

# Save the figure
plt.savefig("Humidity Heatmap.png")

#display figure
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#filter files with zero cloudiness
vacation_no_cloud=dropna_weather_data[dropna_weather_data["Cloudiness"]==0]
#filter files with above max_temp above 75
vacation_above_75degreed=vacation_no_cloud[vacation_no_cloud["Max Temp"]>70]
#filter files with below max_temp below 80
vacation_below_45degreed=vacation_above_75degreed[vacation_above_75degreed["Max Temp"]<80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_45degreed[vacation_below_45degreed["Wind Speed"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity"]<60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,castro,-24.7911,-50.0119,70.02,40,0,7.74,BR,1627139235
1,arraial do cabo,-22.8362,-46.6460,76.89,22,0,5.86,BR,1627139238
2,esmeraldas,-19.7625,-44.3139,70.66,35,0,8.08,BR,1627139256
3,cidreira,-30.1811,-50.2056,70.47,49,0,2.44,BR,1627139290
4,sarkand,45.4100,79.9186,76.33,32,0,5.53,KZ,1627139297
5,southbridge,42.0751,-72.0334,79.02,58,0,4.00,US,1627139304
6,mairinque,-23.5458,-47.1833,74.12,29,0,5.93,BR,1627139306
7,namibe,-15.1961,12.1522,74.52,55,0,7.61,AO,1627139116
8,miandrivazo,-19.5167,45.4667,76.95,37,0,3.69,MG,1627139338
9,altay,47.8667,88.1167,70.47,44,0,7.47,CN,1627139083


In [37]:
#locations
vacation_locations=indexed_perfect_vacation[['Lat', 'Lng']]
vacation_humidity=indexed_perfect_vacation['Humidity'].astype(float)

# Plot Heatmap
vacation_fig=gmaps.figure()

#create Heat Layer
vacation_heat_layer=gmaps.heatmap_layer(vacation_locations, weights=vacation_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

#Add Layer
vacation_fig.add_layer(vacation_heat_layer)

# Save the figure
plt.savefig("vacation_fig.png")

#display figure
vacation_fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
indexed_perfect_vacation.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,castro,-24.7911,-50.0119,70.02,40,0,7.74,BR,1627139235
1,arraial do cabo,-22.8362,-46.6460,76.89,22,0,5.86,BR,1627139238
2,esmeraldas,-19.7625,-44.3139,70.66,35,0,8.08,BR,1627139256
3,cidreira,-30.1811,-50.2056,70.47,49,0,2.44,BR,1627139290
4,sarkand,45.4100,79.9186,76.33,32,0,5.53,KZ,1627139297


In [49]:
# Hotel variable
hotels = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Lat"]
    lng = indexed_perfect_vacation.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    
    # assemble url and make API request
    hotel_response = requests.get(base_url, params=params).json()
    #hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
0,castro,-24.7911,-50.0119,70.02,40,0,7.74,BR,1627139235,CHACARA BAILLY
1,arraial do cabo,-22.8362,-46.6460,76.89,22,0,5.86,BR,1627139238,Pousada da Maricota
2,esmeraldas,-19.7625,-44.3139,70.66,35,0,8.08,BR,1627139256,Bom Será - Marcinho e Fernanda
3,cidreira,-30.1811,-50.2056,70.47,49,0,2.44,BR,1627139290,Hotel Castelo
4,sarkand,45.4100,79.9186,76.33,32,0,5.53,KZ,1627139297,Гостиница Сарканд
5,southbridge,42.0751,-72.0334,79.02,58,0,4.00,US,1627139304,Southbridge Hotel & Conference Center
6,mairinque,-23.5458,-47.1833,74.12,29,0,5.93,BR,1627139306,Sorocabana Palace Hotel
7,namibe,-15.1961,12.1522,74.52,55,0,7.61,AO,1627139116,Hotel Chik Chik Namibe
8,miandrivazo,-19.5167,45.4667,76.95,37,0,3.69,MG,1627139338,Hôtel Arc En Ciel Miandrivazo
9,altay,47.8667,88.1167,70.47,44,0,7.47,CN,1627139083,Jinqiao Hotel


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Lat", "Lng"]]
locations

,Lat,Lng
0,-24.7911,-50.0119
1,-22.8362,-46.6460
2,-19.7625,-44.3139
3,-30.1811,-50.2056
4,45.4100,79.9186
5,42.0751,-72.0334
6,-23.5458,-47.1833
7,-15.1961,12.1522
8,-19.5167,45.4667
9,47.8667,88.1167


In [54]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vacation_fig.add_layer(markers)

# Save the figure
plt.savefig("Nearest Hotel.png")

# Display figure
vacation_fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>